In [ ]:
# για το scraping
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
# σύνδεση με το gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# βασικό url του site
main_url = 'https://www.newsbomb.gr'
# αρχικό url των teaser pages
start_url = 'https://www.newsbomb.gr/ellada?page='

start_page = 2
end_page = 20

In [ ]:
news_list = []

for page_num in range(start_page, end_page+1):
  page_url = start_url + str(page_num)
  response = requests.get(page_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo teaser_articles_list***
  articles_list = doc.find('div', {'class': 'p10'}).find_all('div', {'class': 'cd card-item md-flat t-22 md-t-20 has-img ctype-story'})


  for teaser in articles_list:
      story_dict = {}
      try:

        #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στο link, ΜΕΤΑ το teaser
        link = teaser.find('a', {'class': ''})['href']


        if link.startswith("http"):
          story_dict['url'] = link
        else:
          story_dict['url'] = main_url + link
      except:
        story_dict['url'] = ''
        pass
      news_list.append(story_dict)

In [ ]:
news_bomb_df = pd.DataFrame(news_list)
# news_bomb_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Copy of Copy of Homework 8-news_bomb_csv',index=False)
news_bomb_df

,url
0,https://www.newsbomb.gr/ellada/story/1457679/p...
1,https://www.newsbomb.gr/ellada/story/1457675/k...
2,https://www.newsbomb.gr/ellada/story/1457674/a...
3,https://www.newsbomb.gr/ellada/story/1457673/n...
4,https://www.newsbomb.gr/ellada/astynomiko-repo...
...,...
469,https://www.newsbomb.gr/ellada/story/1456505/s...
470,https://www.newsbomb.gr/ellada/story/1456470/p...
471,https://www.newsbomb.gr/ellada/story/1456504/e...
472,https://www.newsbomb.gr/ellada/story/1456502/s...


In [ ]:
full_articles_list = []

for article_url in news_bomb_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo article που βρήκατε παραπάνω***
  article = doc.find('article')
  full_article_dict = {}

  # website
  full_article_dict['site'] = "newsbomb.gr"

  # url
  full_article_dict['url'] = article_url

  #section
  try:
    section = doc.find('a', {'class': 'main-category'}).text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  #title
  try:
    title = article.find('h1').text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  # date
  try:
    date = article.find('time')['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass


  # author
  try:
    author = article.find('p', {'class', "main-author story-author"}).span.text
    full_article_dict['author'] = author
  except:
    full_article_dict['author'] = ''
    pass

  # full_text
  try:
    p_texts_list = []
    paragraphs = article.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  # media
  media = article.find('figure', {'class', 'media-figure'}) or article.find('div', {'class', 'embed-video'})
  if media:
    full_article_dict['Media'] = True
  else:
    full_article_dict['Media'] = False

  full_articles_list.append(full_article_dict)

In [ ]:
news_bomb_df = pd.DataFrame(full_articles_list)
news_bomb_df

,site,url,section,title,date,author,full_text,Media
0,newsbomb.gr,https://www.newsbomb.gr/ellada/story/1457679/p...,ΕΛΛΑΔΑ,Πυροβολισμοί σε κλαμπ στο κέντρο της Αθήνας,2023-08-12T12:32:02+03:00,Newsbomb,Newsbomb Πυροβολισμοί ακούστηκαν λίγο μετά τις...,True
1,newsbomb.gr,https://www.newsbomb.gr/ellada/story/1457675/k...,ΕΛΛΑΔΑ,Κιλκίς: Αλβανοί μπήκαν στη χώρα με σάκους που ...,2023-08-12T12:22:03+03:00,Newsbomb,Newsbomb Κιλκίς: Οι συλληφθέντες στην θέα των ...,False
2,newsbomb.gr,https://www.newsbomb.gr/ellada/story/1457674/a...,ΕΛΛΑΔΑ,Ασθενής αυτοκτόνησε στο Αττικόν Νοσοκομείο,2023-08-12T12:10:08+03:00,Newsbomb,Newsbomb Ένα απίστευτο περιστατικό έλαβε χώρα ...,False
3,newsbomb.gr,https://www.newsbomb.gr/ellada/story/1457673/n...,ΕΛΛΑΔΑ,Νέα Φιλαδέλφεια: Στον ανακριτή ο αδερφός του Μ...,2023-08-12T12:00:22+03:00,,Σοφία Σπίγγου Την απόδοση Δικαιοσύνης για το θ...,True
4,newsbomb.gr,https://www.newsbomb.gr/ellada/astynomiko-repo...,ΑΣΤΥΝΟΜΙΚΟ,Πάτρα: Συνελήφθη 50χρονος κατηγορούμενος για α...,2023-08-12T11:49:07+03:00,Newsbomb,Newsbomb Πάτρα: Συνελήφθη 50χρονος κατηγορούμε...,False
...,...,...,...,...,...,...,...,...
469,newsbomb.gr,https://www.newsbomb.gr/ellada/story/1456505/s...,ΕΛΛΑΔΑ,Συγκλονιστικό σκίτσο: Άλκης Καμπανός και Μιχάλ...,2023-08-08T23:03:48+03:00,Newsbomb,Newsbomb Η δολοφονία του Μιχάλη Κατσούρη στη Ν...,False
470,newsbomb.gr,https://www.newsbomb.gr/ellada/story/1456470/p...,ΕΛΛΑΔΑ,Προκήρυξη της Σχολής Εθνικής Άμυνας για την πρ...,2023-08-08T22:55:27+03:00,Newsbomb,Newsbomb Οι ενδιαφερόμενοι-ες που επιθυμούν να...,False
471,newsbomb.gr,https://www.newsbomb.gr/ellada/story/1456504/e...,ΕΛΛΑΔΑ,Ερντογάν: Δεν έχουμε προβλήματα με τους γείτον...,2023-08-08T22:52:45+03:00,Newsbomb,Newsbomb Ο Ρετζέπ Ταγίπ Ερντογάν δήλωσε πως η ...,False
472,newsbomb.gr,https://www.newsbomb.gr/ellada/story/1456502/s...,ΕΛΛΑΔΑ,"Σεισμός 3,8 Ρίχτερ στη Βάθεια Λακωνίας",2023-08-08T22:48:00+03:00,Newsbomb,Newsbomb Σεισμός στη Λακωνία Σεισμική δόνηση τ...,True


In [ ]:
news_bomb_df.to_csv('drive/MyDrive/newsbomb.csv', index=False)